## CRUD Functions

First we look at basic CRUD operations. In Jina, CRUD corresponds to four functions: `index` (create), `search` (read), `update`, and `delete`. With documents below as example:


In [1]:
#Run this command to install Jina in this notebook
!pip install jina[hub]

In [2]:
import numpy as np
from jina import Document

docs = [Document(id='🐲', embedding=np.array([0, 0]), tags={'guardian': 'Azure Dragon', 'position': 'East'}),
        Document(id='🐦', embedding=np.array([1, 0]), tags={'guardian': 'Vermilion Bird', 'position': 'South'}),
        Document(id='🐢', embedding=np.array([0, 1]), tags={'guardian': 'Black Tortoise', 'position': 'North'}),
        Document(id='🐯', embedding=np.array([1, 1]), tags={'guardian': 'White Tiger', 'position': 'West'})]

Let's build a simple Flow with a simple indexer.

In [3]:
from jina import Flow
f = Flow().add(uses='jinahub+docker://NumpyPostgresSearcher')

`Document` and `Flow` are important concepts in Jina, which will be explained later. [NumpyPostgresSearcher](https://github.com/jina-ai/executor-indexers/tree/main/jinahub/indexers/searcher/compound/NumpyPostgresSearcher) is a built-in [compound vector-based searcher](https://github.com/jina-ai/executor-indexers#jina-indexers) that stores embedding and structured info for similarity search.


### Indexing

This will store four docs (both embedding and structured info) into Jina.

In [4]:
with f:
    f.index(docs, on_done=print)

/Users/candice/miniconda3/lib/python3.9/site-packages/rich/live.py:227: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')


           pod0@7645[I]:usage: jina [-h] [-v] [-vf]
           pod0@7645[I]:{hello, executor, pod, flow ... 6 more choices} ...
           pod0@7645[I]:jina: error: unrecognized arguments: --runs-in-docker
           pod0@7645[E]:Exception('the container fails to start, check the arguments or entrypoint') during <class 'jina.peapods.runtimes.container.ContainerRuntime'> initialization
 add "--quiet-error" to suppress the exception details
Traceback (most recent call last):
  File "/Users/candice/miniconda3/lib/python3.9/site-packages/jina/peapods/peas/__init__.py", line 78, in run
    runtime = runtime_cls(
  File "/Users/candice/miniconda3/lib/python3.9/site-packages/jina/peapods/runtimes/container/__init__.py", line 30, in __init__
    raise Exception(
Exception: the container fails to start, check the arguments or entrypoint
           Flow@7634[E]:pod0:<jina.peapods.pods.Pod object at 0x7f896d868c70> can not be started due to RuntimeFailToStart(), Flow is aborted


RuntimeFailToStart: 

### Searching

This will print 🐲🐦🐢 Doc with score 0, 1, 1, respectively, as these three documents are nearest neighbout of the query document.

In [ ]:
with f:
    f.search(docs[0], top_k=3,
             on_done=lambda x: [print(m.dict()) for m in x.docs[0].matches])

### Updating

Update 🐲 embedding to `[1, 1]`

In [ ]:
docs[0].embedding = np.array([1, 1])
with f:
    f.update(docs[0])


### Deleting

Remove 🐦🐲 Documents

In [ ]:
with f:
    f.delete(['🐦', '🐲'])